__Домашняя работа к уроку №8__

Взять датасет из google диска: https://drive.google.com/file/d/1MpWBFIbqu4mbiD0BBKYX6YhS-f4mN3Z_. Проверить гипотезу о том, в каком варианте теста (control/personalization) больше конверсия (converted) и значимо ли это отличие статистически.

In [21]:
import pandas as pd
import numpy as np

In [1]:
!wget 'https://drive.google.com/uc?export=download&id=1MpWBFIbqu4mbiD0BBKYX6YhS-f4mN3Z_' -O data.zip

--2022-04-18 12:51:42--  https://drive.google.com/uc?export=download&id=1MpWBFIbqu4mbiD0BBKYX6YhS-f4mN3Z_
Resolving drive.google.com (drive.google.com)... 172.217.1.206, 2607:f8b0:4004:801::200e
Connecting to drive.google.com (drive.google.com)|172.217.1.206|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-04-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/569foi0t65bm46oh29v8ed4oott2okp7/1650286275000/14904333240138417226/*/1MpWBFIbqu4mbiD0BBKYX6YhS-f4mN3Z_?e=download [following]
--2022-04-18 12:51:43--  https://doc-04-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/569foi0t65bm46oh29v8ed4oott2okp7/1650286275000/14904333240138417226/*/1MpWBFIbqu4mbiD0BBKYX6YhS-f4mN3Z_?e=download
Resolving doc-04-c0-docs.googleusercontent.com (doc-04-c0-docs.googleusercontent.com)... 142.251.33.193, 2607:f8b0:4004:837::2001
Connecting to doc-04-c0-docs.googleusercontent.com (doc-04-c0-docs.googleuser

In [2]:
!unzip data.zip

Archive:  data.zip
  inflating: marketing description.txt  
  inflating: marketing_campaign.csv  
  inflating: subscribers.csv         
  inflating: users.csv               


In [9]:
df = pd.read_csv('marketing_campaign.csv')
df.head()

,user_id,date_served,marketing_channel,variant,language_displayed,converted
0,a1000,1/1/18,House Ads,personalization,English,True
1,a1001,1/1/18,House Ads,personalization,English,True
2,a1002,1/1/18,House Ads,personalization,English,True
3,a1003,1/1/18,House Ads,personalization,English,True
4,a1004,1/1/18,House Ads,personalization,English,True


Посмотрим на группы отдельно

In [11]:
df[df['variant'] == 'personalization'].describe()

,user_id,date_served,marketing_channel,variant,language_displayed,converted
count,4946,4945,4946,4946,4946,4946
unique,4946,31,5,1,4,2
top,a1000,1/15/18,House Ads,personalization,English,False
freq,1,434,2181,4946,4863,4241


In [14]:
df[df['variant'] == 'control'].describe()

,user_id,date_served,marketing_channel,variant,language_displayed,converted
count,5091,5076,5076,5091,5091,5076
unique,5091,31,5,1,4,2
top,a1055,1/15/18,House Ads,control,English,False
freq,1,355,2552,5091,4930,4705


Вычислим конверсии в каждой группе.

In [13]:
df.loc[df['variant']=='personalization']['converted'].value_counts()

False    4241
True      705
Name: converted, dtype: int64

In [15]:
con_personalization = 705/(4241+705)
con_personalization

0.14253942579862516

In [17]:
df.loc[df['variant']=='control']['converted'].value_counts()

False    4705
True      371
Name: converted, dtype: int64

In [18]:
con_control = 371/(4705+371)
con_control

0.0730890464933018

Видно, что конверсия в группе с персонализированной рекламой больше почти в 2 раза, чем в контрольной группе. Теперь необходимо определить статичтиескую значимостьэтой разницы.

Так как рассматриваемый показатель является бинарным, то используем для этого Z-критерий.

In [19]:
k1 = 705
n1 = 4241+705
k2 = 371
n2 = 4705+371

Выдвинем нулевую гипотезу о том, что конверсии в обеих группах равны.

In [22]:
from statsmodels.stats import proportion

z_score, z_pvalue = proportion.proportions_ztest(np.array([k1, k2]), 
                                                 np.array([n1, n2]))

print(f'Results are z_score={z_score:.3f} pvalue={z_pvalue:.3f}')

Results are z_score=11.228 pvalue=0.000


Так как pvalue меньше 0.05, то мы отвергаем нулевую гипотезу. Конверсии в группах значимо отличаются. 

Такой же результат получаем для непараметрического критерия хи квадрат.

In [23]:
chisq, pvalue, table = proportion.proportions_chisquare(np.array([k1, k2]), 
                                                        np.array([n1, n2]))

print(f'Results are chisq={chisq:.3f} pvalue={pvalue:.3f}')

Results are chisq=126.078 pvalue=0.000
